In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [3]:
int_stop = pd.read_csv(simplify_dir + 'int_stop.csv')

In [4]:
int_stop = gpd.GeoDataFrame(
    int_stop, geometry=gpd.points_from_xy(int_stop.int_lon, int_stop.int_lat))

In [5]:
wards = gpd.read_file('GIS/City Wards Data.geojson')

In [6]:
int_stop = int_stop.set_crs(epsg = '4326')

In [7]:
int_stop

,INT_ID,int_lon,int_lat,geometry
0,110,-79.396989,43.697748,POINT (-79.39699 43.69775)
1,205,-79.475991,43.652048,POINT (-79.47599 43.65205)
2,214,-79.403889,43.667249,POINT (-79.40389 43.66725)
3,208,-79.452723,43.657023,POINT (-79.45272 43.65702)
4,115,-79.412755,43.767867,POINT (-79.41276 43.76787)
...,...,...,...,...
3527,13460349,-79.363253,43.687296,POINT (-79.36325 43.68730)
3528,13459088,-79.273038,43.692872,POINT (-79.27304 43.69287)
3529,13456297,-79.265758,43.709715,POINT (-79.26576 43.70971)
3530,13441958,-79.175723,43.818247,POINT (-79.17572 43.81825)


In [8]:
int_stop_buffer = int_stop.copy().to_crs(epsg = '32617')

In [9]:
int_stop_buffer['geometry'] = int_stop_buffer.buffer(50)

In [10]:
int_wards = gpd.sjoin(wards.to_crs(epsg = '32617'), int_stop_buffer, op = 'intersects', how = 'right')[['AREA_NAME', 'AREA_SHORT_CODE', 'INT_ID']]

In [11]:
int_wards

,AREA_NAME,AREA_SHORT_CODE,INT_ID
0,Toronto-St. Paul's,12,110
1,Parkdale-High Park,4,205
2,University-Rosedale,11,214
3,Parkdale-High Park,4,208
4,Willowdale,18,115
...,...,...,...
3527,University-Rosedale,11,13460349
3528,Scarborough Southwest,20,13459088
3529,Scarborough Southwest,20,13456297
3530,Scarborough-Rouge Park,25,13441958


In [12]:
#int_stop[int_stop['INT_ID'].isin(list(int_wards[int_wards['AREA_SHORT_CODE'].isna()]['INT_ID']))].plot()

In [13]:
#int_stop[int_stop['INT_ID'].isin(list(int_wards[int_wards['AREA_SHORT_CODE'].isna()]['INT_ID']))].to_file('GIS/steeles.geojson', driver = 'GeoJSON')

In [14]:
int_wards

,AREA_NAME,AREA_SHORT_CODE,INT_ID
0,Toronto-St. Paul's,12,110
1,Parkdale-High Park,4,205
2,University-Rosedale,11,214
3,Parkdale-High Park,4,208
4,Willowdale,18,115
...,...,...,...
3527,University-Rosedale,11,13460349
3528,Scarborough Southwest,20,13459088
3529,Scarborough Southwest,20,13456297
3530,Scarborough-Rouge Park,25,13441958


In [15]:
int_wards['AREA_SHORT_CODE'] = np.where(int_wards['INT_ID'].isin([13464485]),
                    2, int_wards['AREA_SHORT_CODE'])

In [16]:
int_wards['AREA_SHORT_CODE'] = np.where(int_wards['INT_ID'].isin([13463579,13449352]),
                    1, int_wards['AREA_SHORT_CODE'])

In [17]:
int_wards['AREA_SHORT_CODE'] = np.where(int_wards['INT_ID'].isin([13448601,13447821,13446751,13446566,30103926,13446357]),
                    7, int_wards['AREA_SHORT_CODE'])

In [18]:
int_wards['AREA_SHORT_CODE'] = np.where(int_wards['INT_ID'].isin([13445817,13445306,13444956]),
                    6, int_wards['AREA_SHORT_CODE'])

In [19]:
int_wards['AREA_SHORT_CODE'] = np.where(int_wards['INT_ID'].isin([13444464,13444337]),
                    18, int_wards['AREA_SHORT_CODE'])

In [20]:
int_wards['AREA_SHORT_CODE'] = np.where(int_wards['INT_ID'].isin([13442003,13441903]),
                    22, int_wards['AREA_SHORT_CODE'])

In [21]:
int_wards['AREA_SHORT_CODE'] = np.where(int_wards['INT_ID'].isin([13441706,13441677,13441616]),
                    23, int_wards['AREA_SHORT_CODE'])

In [22]:
int_wards['AREA_SHORT_CODE'] = int_wards['AREA_SHORT_CODE'].astype(int)

In [23]:
int_wards.to_csv('GIS/int_wards.csv', index = False)

In [24]:
int_stop[['INT_ID', 'geometry']].merge(int_wards).to_file('GIS/int_wards.geojson', driver = 'GeoJSON')